## Ejemplos del Linear Chain CRF

### Nombre

* linearCRF_l2_zero_50_0.1_0_k_10.crfsuite

### Parametros

* l1 = 0.1
* l2 = 0
* Precisión general = 0.9516
* Max_Iter = 50

In [1]:
import os
import pycrfsuite
import sys  
import random
sys.path.insert(0, '../')
from utils import get_corpus, WordsToLetter, extractLabels, extractTokens
model = "linearCRF_l2_zero_50_0.1_0_k_10.crfsuite"

In [2]:
model_path = os.path.join("../models/linearChainCRFs",
                          "tsu_" + model)
print(model_path)
tagger = pycrfsuite.Tagger()
# Cargando modelos preentrenados
tagger.open(model_path)

../models/linearChainCRFs/tsu_linearCRF_l2_zero_50_0.1_0_k_10.crfsuite


In [3]:
# Obteniendo corpus para pruebas
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + get_corpus('corpus_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)

In [4]:
# funciones auxiliares
def obtener_palabras(frases):
    palabras = []
    for frase in frases:
        chunks = [palabra[0] for palabra in frase[:-1]]
        palabras.append("".join(chunks))
    return palabras

def reporte(prediction_tags, real_tags, example):
    print("Letra | Predicción | Real | Es correcto?")
    for prediction, real, letter in zip(prediction_tags, real_tags, extractTokens(example)):
        print(f"{letter} | {prediction} | {real} | {True if prediction == real else False}")
        
def accuracy_score(y_test, y_pred):
    right, total = 0, 0
    for real, prediction in zip(y_test, y_pred):
        if real == prediction:
            right += 1
    return right / len(y_test)

In [5]:
def feature_functions_maker(sent):
    ''' Reglas que configuran las feature functions para entrenamiento

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''

    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                # Position of word in sentence
                if i == senlen -1:
                    features.append("EOS")
                else:
                    features.append("BOS")

                # Pos tag sequence (Don't get pos tag if sentence is 1 word long)
                if i > 0 and senlen > 1:
                    features.append('prevpostag=' + sent[i-1][0][1])
                    if i != senlen-1:
                        features.append('nxtpostag=' + sent[i+1][0][1])

                # Position of letter in word
                if j == 0:
                    features.append('BOW')
                elif j == wordlen-1:
                    features.append('EOW')
                else:
                    features.append('letterposition=-%s' % str(wordlen-1-j))

                # Letter sequences before letter
                if j >= 4:
                    features.append('prev4letters=' + lettersequence[j-4:j].lower() + '>')
                if j >= 3:
                    features.append('prev3letters=' + lettersequence[j-3:j].lower() + '>')
                if j >= 2:
                    features.append('prev2letters=' + lettersequence[j-2:j].lower() + '>')
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')

                # letter sequences after letter
                if j <= wordlen-2:
                    nxtlets = word[j+1][0]
                    features.append('nxtletter=<' + nxtlets.lower())
                if j <= wordlen-3:
                    nxtlets += word[j+2][0]
                    features.append('nxt2letters=<' + nxtlets.lower())
                if j <= wordlen-4:
                    nxtlets += word[j+3][0]
                    features.append('nxt3letters=<' + nxtlets.lower())
                if j <= wordlen-5:
                    nxtlets += word[j+4][0]
                    features.append('nxt4letters=<' + nxtlets.lower())

            # Add encoding for pysrfsuite
            featurelist.append([f.encode('utf-8') for f in features])
    return featurelist

## Los peores ejemplos

In [6]:
peores = []
for i, example in enumerate(letter_corpus):
    feature_functions = feature_functions_maker(example)    
    prediction_tags = tagger.tag(feature_functions)      
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy <= 0.8:
        peores.append((i, accuracy))
peores = sorted(peores, key=lambda t: t[1])

In [10]:
# Tomamos los 5 peores
top = peores[5:10]
for i, data in enumerate(top):
    print("-"*50)
    index = data[0]
    palabras = obtener_palabras(corpus[index])
    example = letter_corpus[index]
    features = feature_functions_maker(example)
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    print(f"Ejemplo {i+1} de 5 | Frase: \"{' '.join(palabras)}\" | Accuracy: {data[1]}")
    reporte(prediction_tags, real_tags, example)

--------------------------------------------------
Ejemplo 1 de 5 | Frase: "ibé" | Accuracy: 0.3333333333333333
Letra | Predicción | Real | Es correcto?
i | B-stem | B-3.icp | False
b | I-stem | B-stem | False
é | I-stem | I-stem | True
--------------------------------------------------
Ejemplo 2 de 5 | Frase: "bindμzú" | Accuracy: 0.42857142857142855
Letra | Predicción | Real | Es correcto?
b | B-3.cpl | B-3.cpl | True
i | I-3.cpl | I-3.cpl | True
n | B-stem | B-muy | False
d | I-stem | I-muy | False
μ | I-stem | I-muy | False
z | I-stem | B-stem | False
ú | I-stem | I-stem | True
--------------------------------------------------
Ejemplo 3 de 5 | Frase: "giné" | Accuracy: 0.5
Letra | Predicción | Real | Es correcto?
g | B-psd | B-2.pot | False
i | B-3.icp | I-2.pot | False
n | B-stem | B-stem | True
é | I-stem | I-stem | True
--------------------------------------------------
Ejemplo 4 de 5 | Frase: "takhμxyι' xogígé" | Accuracy: 0.6
Letra | Predicción | Real | Es correcto?
t | B-has

### Los mejores

Como de estos hay muchos vamos a tomar 3 al azar

In [8]:
flag = 0
while flag != 3:
    print("-"*50)
    index = random.randint(0, len(letter_corpus))
    palabras = obtener_palabras(corpus[index])
    example = letter_corpus[index]
    features = feature_functions_maker(example)
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy >= 0.99:
        print(f"Ejemplo {flag + 1} de 3 | Frase: \"{' '.join(palabras)}\" | Precisión: {accuracy}")
        reporte(prediction_tags, real_tags, example)
        flag += 1

--------------------------------------------------
Ejemplo 1 de 3 | Frase: "mbrιndμ'μ mäñá" | Precisión: 1.0
Letra | Predicción | Real | Es correcto?
m | B-psd | B-psd | True
b | B-lig | B-lig | True
r | B-3.cnt | B-3.cnt | True
ι | I-3.cnt | I-3.cnt | True
n | B-muy | B-muy | True
d | I-muy | I-muy | True
μ | I-muy | I-muy | True
' | B-stem | B-stem | True
μ | I-stem | I-stem | True
m | B-1.pss | B-1.pss | True
ä | I-1.pss | I-1.pss | True
ñ | B-stem | B-stem | True
á | I-stem | I-stem | True
--------------------------------------------------
Ejemplo 2 de 3 | Frase: "gogwádi gohμté" | Precisión: 1.0
Letra | Predicción | Real | Es correcto?
g | B-1.pot | B-1.pot | True
o | I-1.pot | I-1.pot | True
g | B-stem | B-stem | True
w | I-stem | I-stem | True
á | I-stem | I-stem | True
d | I-stem | I-stem | True
i | I-stem | I-stem | True
g | B-1.pot | B-1.pot | True
o | I-1.pot | I-1.pot | True
h | B-stem | B-stem | True
μ | I-stem | I-stem | True
t | I-stem | I-stem | True
é | I-stem | I-stem

In [13]:
# Obteniendo las frases mas largas y mejor etiquetadas
min_len = 30
phrases = []
for i, example in enumerate(letter_corpus):
    features = feature_functions_maker(example)
    palabras = obtener_palabras(corpus[i])
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy >= 0.99:
        phrase = " ".join(palabras)
        if len(phrase) >= min_len:
            phrases.append((palabras, accuracy, i))
print(phrases[:3])

[(['ndóphμdi', 'dópεphí', "bit'μngí", 'bimähtratágí', 'ko', 'chíkóhté'], 1.0, 0), (["bi'μngí", 'yι', 'mbμhí', 'nge', 'hín', 'dímáné', 'gwaporá', 'nge', 'dímádáhní'], 1.0, 1), (['ya', 'hín', 'dóné', 'dómμikó', 'komo', "mi'μngí"], 1.0, 5)]


In [14]:
import csv

with open("mejores-frases-linearCRF.csv", "w") as csvfile:
    header = ["frase", "glosa", 'accuracy-score']
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for phrase in phrases:
        writer.writerow([" ".join(phrase[0]), corpus[phrase[2]], str(phrase[1])])

In [18]:
# Obteniendo las frases mas largas y mejor etiquetadas
min_len = 0
phrases = []
for i, example in enumerate(letter_corpus):
    features = feature_functions_maker(example)
    palabras = obtener_palabras(corpus[i])
    prediction_tags = tagger.tag(features)
    real_tags = extractLabels(example, 1)
    accuracy = accuracy_score(real_tags, prediction_tags)
    if accuracy <= 0.80:
        phrase = " ".join(palabras)
        if len(phrase) >= min_len:
            phrases.append((palabras, accuracy, i))
print(phrases[:3])

[(['bianda', 'pobrehí'], 0.6923076923076923, 60), (['mihtsú'], 0.3333333333333333, 94), (['té', 'gothó', 'mikhá', 'níwimá'], 0.7777777777777778, 134)]


In [16]:
import csv

with open("peores-frases-linearCRF.csv", "w") as csvfile:
    header = ["frase", "glosa predicha", "glosa real", 'accuracy-score']
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for phrase in phrases:
        writer.writerow([" ".join(phrase[0]), corpus[phrase[2]], str(phrase[1])])